In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import string # special operations on strings
import spacy # language models
import re
from wordcloud import WordCloud

Problem-1)
Perform sentimental analysis on the Elon-musk tweets (Elon-musk.csv)

Solution:

In [ ]:
#data loading
from google.colab import files
uploaded=files.upload()

Saving Elon_musk.csv to Elon_musk.csv


In [ ]:
elon_musk=pd.read_csv('Elon_musk.csv',index_col=0,encoding='ISO-8859-1')
elon_musk

,Text
1,@kunalb11 Im an alien
2,@ID_AA_Carmack Ray tracing on Cyberpunk with H...
3,@joerogan @Spotify Great interview!
4,@gtera27 Doge is underestimated
5,@teslacn Congratulations Tesla China for amazi...
...,...
1995,"@flcnhvy True, it sounds so surreal, but the n..."
1996,@PPathole Make sure to read ur terms &amp; con...
1997,@TeslaGong @PPathole Samwise Gamgee
1998,@PPathole Altho Dumb and Dumber is <U+0001F525...


Basic & EDA

In [ ]:
elon_musk.shape

(1999, 1)

In [ ]:
elon_musk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1999 entries, 1 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    1999 non-null   object
dtypes: object(1)
memory usage: 31.2+ KB


In [ ]:
elon_musk.isna().sum()

Text    0
dtype: int64

* There is no null values in dataset

In [ ]:
elon_musk.duplicated().value_counts()

False    1997
True        2
dtype: int64

* there are 2 duplicated data in dataset

In [ ]:
elon_musk[elon_musk.duplicated()]

,Text
325,The most entertaining outcome is the most likely
656,@westcoastbill Indeed


In [ ]:
#removing duplicate records
elon_musk_cln=elon_musk.drop_duplicates().reset_index()
elon_musk_cln=elon_musk_cln.drop('index',axis=1)
elon_musk_cln

,Text
0,@kunalb11 Im an alien
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...
2,@joerogan @Spotify Great interview!
3,@gtera27 Doge is underestimated
4,@teslacn Congratulations Tesla China for amazi...
...,...
1992,"@flcnhvy True, it sounds so surreal, but the n..."
1993,@PPathole Make sure to read ur terms &amp; con...
1994,@TeslaGong @PPathole Samwise Gamgee
1995,@PPathole Altho Dumb and Dumber is <U+0001F525...


In [ ]:
elon_musk_cln.value_counts()
     

Text                                                                                                                                            
.@TheBabylonBee is savage <U+0001F923><U+0001F923>                                                                                                  1
@picot_john @CyberpunkGame Its a good to great game imo, but improving what may seem like mere surface detail woul
 https://t.co/zm0SZ9CbZg        1
@renarddemoscou @BillGates Sorry to hear this. What can we do to help?                                                                              1
@reganlovetesla @RenataKonkoly @SpaceNews_Inc Enough already on my plate &amp; Teslas. Maybe one day will do a prototy
 https://t.co/tGPWEJ7WQA    1
@redmercy Yes                                                                                                                                       1
                                                                                                         

* Since we have only one feature and we removed duplicate observations, there will be only unique observations in the dataset.

Text cleaning

In [ ]:
elon_musk_cln.head()

,Text
0,@kunalb11 Im an alien
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...
2,@joerogan @Spotify Great interview!
3,@gtera27 Doge is underestimated
4,@teslacn Congratulations Tesla China for amazi...


In [ ]:
#regular expression
import re 
import string

def clean_text(text):
  text=text.lower()
  text=re.sub('\[.*?\']','',text)
  text=re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text=re.sub('\w*\d\w*','',text)
  text=re.sub("[0-9" "]+"," ",text)
  text=re.sub('[‘’“”…]','',text)
  text=re.sub(" ’ ",'',text)
  return text

clean=lambda x:clean_text(x)

In [ ]:
elon_musk_cln["Text"] = elon_musk_cln["Text"].apply(clean_text) # Clean The Text
elon_musk_cln

,Text
0,im an alien
1,idaacarmack ray tracing on cyberpunk with hdr ...
2,joerogan spotify great interview
3,doge is underestimated
4,teslacn congratulations tesla china for amazin...
...,...
1992,flcnhvy true it sounds so surreal but the nega...
1993,ppathole make sure to read ur terms amp condit...
1994,teslagong ppathole samwise gamgee
1995,ppathole altho dumb and dumber is


In [ ]:
#word frequency
#top 20
freq=pd.Series(''.join(elon_musk_cln['Text']).split()).value_counts()[:25]
freq

the               473
to                399
is                352
a                 351
of                312
spacex            226
in                219
amp               218
for               194
will              163
tesla             159
be                155
we                133
this              132
on                128
that              127
it                126
but               125
i                 114
with              107
are                99
erdayastronaut     98
at                 93
you                85
and                84
dtype: int64

In [ ]:
#removing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
elon_musk_cln['Text']=elon_musk_cln['Text'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))
elon_musk_cln

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Text
0,im alien
1,idaacarmack ray tracing cyberpunk hdr nextleve...
2,joerogan spotify great interview
3,doge underestimated
4,teslacn congratulations tesla china amazing ex...
...,...
1992,flcnhvy true sounds surreal negative propagand...
1993,ppathole make sure read ur terms amp condition...
1994,teslagong ppathole samwise gamgee
1995,ppathole altho dumb dumber
